Gina Martínez López

El objetivo del proyecto es usar una red neuronal para clasificar datos sobre señales de
voz, de acuerdo al sentimiento de dicha voz: enojado, triste, feliz.
    

In [1]:
#importar paquetes
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data 
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from sklearn import preprocessing


2.
Se lee el conjunto de datos y se eliminan los datos y columnas innecesarias.

In [2]:
df=pd.read_csv("emotions_by_voice registers.csv")
df[:]

,Unnamed: 0.1,Unnamed: 0,X,meanfreq,sd,median,Q25,Q75,IQR,skew,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0,1,1,0.181338,0.060495,0.187476,0.126197,0.233586,0.107389,0.869088,...,0.181338,0.137742,0.023022,0.271186,0.777344,0.085938,6.226562,6.140625,0.116586,sad
1,1,2,2,0.186897,0.062260,0.195070,0.130847,0.243987,0.113140,1.191767,...,0.186897,0.121811,0.018412,0.271186,0.930339,0.085938,4.000000,3.914062,0.144983,sad
2,2,3,3,0.189102,0.062901,0.204945,0.131422,0.249978,0.118556,1.312690,...,0.189102,0.123758,0.083333,0.262295,0.332386,0.085938,0.625000,0.539062,0.334783,sad
3,4,5,5,0.183036,0.060051,0.174115,0.129949,0.236967,0.107017,1.096409,...,0.183036,0.128469,0.044693,0.258065,1.012019,0.085938,5.468750,5.382812,0.304910,sad
4,5,6,6,0.168793,0.057910,0.156266,0.116783,0.216326,0.099543,1.386837,...,0.168793,0.109720,0.022472,0.235294,0.228795,0.093750,0.750000,0.656250,0.306777,sad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,1243,1244,1436,0.244013,0.035477,0.254385,0.229653,0.265573,0.035920,2.214752,...,0.244013,0.202433,0.028829,0.271186,0.616536,0.210938,1.609375,1.398438,0.281869,happy
905,1244,1245,1437,0.235383,0.045303,0.248974,0.220745,0.264233,0.043488,2.474743,...,0.235383,0.189293,0.031250,0.275862,1.115723,0.265625,5.500000,5.234375,0.167861,happy
906,1245,1246,1438,0.231211,0.044793,0.234847,0.221477,0.262090,0.040613,2.607668,...,0.231211,0.171805,0.022346,0.275862,1.070801,0.265625,4.554688,4.289062,0.214936,happy
907,1246,1247,1439,0.213587,0.082267,0.249435,0.207680,0.268538,0.060858,3.460579,...,0.213587,0.155277,0.020592,0.275862,1.724888,0.273438,6.812500,6.539062,0.238857,happy


In [3]:
#Removemos las filas con valores ausentes.
df=df.dropna()
df

,Unnamed: 0.1,Unnamed: 0,X,meanfreq,sd,median,Q25,Q75,IQR,skew,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0,1,1,0.181338,0.060495,0.187476,0.126197,0.233586,0.107389,0.869088,...,0.181338,0.137742,0.023022,0.271186,0.777344,0.085938,6.226562,6.140625,0.116586,sad
1,1,2,2,0.186897,0.062260,0.195070,0.130847,0.243987,0.113140,1.191767,...,0.186897,0.121811,0.018412,0.271186,0.930339,0.085938,4.000000,3.914062,0.144983,sad
2,2,3,3,0.189102,0.062901,0.204945,0.131422,0.249978,0.118556,1.312690,...,0.189102,0.123758,0.083333,0.262295,0.332386,0.085938,0.625000,0.539062,0.334783,sad
3,4,5,5,0.183036,0.060051,0.174115,0.129949,0.236967,0.107017,1.096409,...,0.183036,0.128469,0.044693,0.258065,1.012019,0.085938,5.468750,5.382812,0.304910,sad
4,5,6,6,0.168793,0.057910,0.156266,0.116783,0.216326,0.099543,1.386837,...,0.168793,0.109720,0.022472,0.235294,0.228795,0.093750,0.750000,0.656250,0.306777,sad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,1243,1244,1436,0.244013,0.035477,0.254385,0.229653,0.265573,0.035920,2.214752,...,0.244013,0.202433,0.028829,0.271186,0.616536,0.210938,1.609375,1.398438,0.281869,happy
905,1244,1245,1437,0.235383,0.045303,0.248974,0.220745,0.264233,0.043488,2.474743,...,0.235383,0.189293,0.031250,0.275862,1.115723,0.265625,5.500000,5.234375,0.167861,happy
906,1245,1246,1438,0.231211,0.044793,0.234847,0.221477,0.262090,0.040613,2.607668,...,0.231211,0.171805,0.022346,0.275862,1.070801,0.265625,4.554688,4.289062,0.214936,happy
907,1246,1247,1439,0.213587,0.082267,0.249435,0.207680,0.268538,0.060858,3.460579,...,0.213587,0.155277,0.020592,0.275862,1.724888,0.273438,6.812500,6.539062,0.238857,happy


In [4]:
#Remuevo las columnas con datos de indices
df=df.drop(df.columns[[0,1,2]], axis=1)
df

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.181338,0.060495,0.187476,0.126197,0.233586,0.107389,0.869088,2.863717,0.923566,0.307220,...,0.181338,0.137742,0.023022,0.271186,0.777344,0.085938,6.226562,6.140625,0.116586,sad
1,0.186897,0.062260,0.195070,0.130847,0.243987,0.113140,1.191767,3.878650,0.918848,0.298859,...,0.186897,0.121811,0.018412,0.271186,0.930339,0.085938,4.000000,3.914062,0.144983,sad
2,0.189102,0.062901,0.204945,0.131422,0.249978,0.118556,1.312690,4.589995,0.919519,0.313069,...,0.189102,0.123758,0.083333,0.262295,0.332386,0.085938,0.625000,0.539062,0.334783,sad
3,0.183036,0.060051,0.174115,0.129949,0.236967,0.107017,1.096409,3.680995,0.921361,0.329295,...,0.183036,0.128469,0.044693,0.258065,1.012019,0.085938,5.468750,5.382812,0.304910,sad
4,0.168793,0.057910,0.156266,0.116783,0.216326,0.099543,1.386837,5.031744,0.926238,0.337047,...,0.168793,0.109720,0.022472,0.235294,0.228795,0.093750,0.750000,0.656250,0.306777,sad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,0.244013,0.035477,0.254385,0.229653,0.265573,0.035920,2.214752,7.565052,0.821874,0.136933,...,0.244013,0.202433,0.028829,0.271186,0.616536,0.210938,1.609375,1.398438,0.281869,happy
905,0.235383,0.045303,0.248974,0.220745,0.264233,0.043488,2.474743,9.959019,0.848109,0.236957,...,0.235383,0.189293,0.031250,0.275862,1.115723,0.265625,5.500000,5.234375,0.167861,happy
906,0.231211,0.044793,0.234847,0.221477,0.262090,0.040613,2.607668,10.698821,0.848702,0.241998,...,0.231211,0.171805,0.022346,0.275862,1.070801,0.265625,4.554688,4.289062,0.214936,happy
907,0.213587,0.082267,0.249435,0.207680,0.268538,0.060858,3.460579,18.034614,0.882544,0.425394,...,0.213587,0.155277,0.020592,0.275862,1.724888,0.273438,6.812500,6.539062,0.238857,happy


Se separa la columna objetivo y se crea el conjunto de entrenamiento, validación y test

In [5]:
##Separar la columna objetivo.
X=df.drop('label',axis=1)
y=df['label']
y.unique()

array(['sad', 'happy', 'angry'], dtype=object)

In [6]:
X=(X-X.mean())/X.std()

Se codifica: happy->0 angry->1 sad->2

In [7]:
le = preprocessing.LabelEncoder()
le.fit(["happy", "angry", "sad"])
y2= le.transform(y) 

El conjunto de datos en atributos X y variable objetivo y. Luego separamos éstos en conjuntos de entrenamiento y testeo, con una proporción de 65-35 para que halla una mejor distribución de los datos.

In [8]:
#Separar en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y2, random_state = 0, test_size=0.35, train_size=0.65, shuffle=True)

Se manejara una validación con proporción 75-25.

In [9]:
#Validación
X_test1, X_val, y_test1, y_val = train_test_split(X_test, y_test, random_state = 0, test_size=0.25, train_size=0.75, shuffle=True)

3. Se preparan los datos para alimentar la red neuronal, primero se hace la conversión a tensores de X y y.

In [10]:
class MyDataset():
 
  def __init__(self,X,y):
    self.X=torch.tensor(X.values,dtype=torch.float32)
    self.y=torch.tensor(y,dtype=torch.long)
 
  def __len__(self):
    return len(self.y)
   
  def __getitem__(self,idx):
    return self.X[idx],self.y[idx]

Ahora usamos los Dataloaders para los conjuntos set, val y test.

In [11]:
# Usar la clase MyDataset para preparar cada conjunto en forma de tensores
train_sec=MyDataset(X_train, y_train)
test_sec=MyDataset(X_test1,y_test1)
val_sec=MyDataset(X_val,y_val)

La creación de dataloaders se usaran para proporcionarlos por lotes a la red

In [12]:
# Definimos los DataLoaders para cargar la información pro lotes
train_data=DataLoader(
    train_sec,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

test_data=DataLoader(
    test_sec,
    batch_size=3,
    shuffle=False,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

val_data=DataLoader(
    val_sec,
    batch_size=3,
    shuffle=False,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

Se imprime el primer bath del Test Set, aqui vemos como con el Dataloader se ingresan los datos a la red neuronal.

In [13]:
for i, (data, labels) in enumerate(test_data):
  print(data.shape, labels.shape)
  print(data,labels)
  break;

torch.Size([3, 20]) torch.Size([3])
tensor([[ 0.0809, -0.9859,  0.1854,  0.3136, -0.6326, -0.7648,  2.0283,  2.2725,
         -0.7538, -0.5520,  0.2844,  0.0809,  0.3691,  0.0123, -0.8960,  1.2587,
          0.3380,  1.3476,  1.3359,  0.0093],
        [-0.1513, -1.0782, -0.1262,  0.1874, -1.2506, -0.9609,  0.9696,  1.1155,
         -0.5804, -0.6929, -0.2459, -0.1513, -0.2692, -0.8255, -0.8960,  1.1099,
          0.2609,  1.3317,  1.3239,  0.2008],
        [ 0.5278, -0.8079,  0.4555,  0.3800, -0.1293, -0.5588, -0.3897, -0.3316,
         -0.1240, -0.7063,  0.2304,  0.5278,  0.2858, -0.1065,  0.8857,  0.1165,
          0.4150, -0.0903, -0.1122, -0.9317]]) tensor([1, 0, 1])


4. 
Se define la clase Net con la arquitectura de la red neuronal, para este caso se usan 2 capas, sin embargo se realizo el experimento con diferentes tamaños de capas, sin embargo el resultado final era menos optimo, por lo tanto, se decidio dejar 2 capas.

In [14]:
class Net(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(Net, self).__init__()
        
        # Inicializar las dos capas lineales 
        self.fc1 = nn.Linear(num_inputs,10)
        self.fc2 = nn.Linear(10, num_outputs)

    def forward(self, x):
      
        # Usar las capas inicializadas y devolver x
        x = self.fc1(x)
        x = self.fc2(x)

        #x = nn.functional.log_softmax(x,dim=1) #Función de activación
        return x

5. 
Se define el modelo, el optimizador con Stochastic gradient descent y un Learning rate de 0.1, luego la función de costo denominada como criterion con la función CrossEntropyLoss la cual se usa para los problemas de clasificación.

In [15]:
model=Net(X_train.shape[1],3)

#TODO 5 Definir el optimizador Stochastic gradient descent y la función MeanSquareError. Usar Learnig rate de 0.1
optimizer=torch.optim.SGD(model.parameters(), lr=0.01)
criterion=torch.nn.CrossEntropyLoss()



 Se entrena el modelo con un num_epochs de 100 suficiente para poder ver el cambio de los errores de entrenamiento y validación, adicionalmente se imprime el accuracy para identificar si hay algún error, pero se identifica que aumenta.

In [16]:
#Entrenar el modelo

def train_model(model,optimizer,loss_module,train_loader,valid_loader,num_epochs=100):
  
  valid_loss_min =np.inf  #Vamos a encontrar el menor valor de error de validación. Por eso la inicializmaos como 'infinito'
  
  for i in range(num_epochs):
    model.train()  #ponemos el modelo en modo entrenamiento. Es importante en otras arquitecturas como redes convolucionales.
    train_loss = 0.0
    v_loss = 0.0
    train_accuracy=0.0
    total = 0

    for data, target in train_loader:

        # Reiniciar los gradientes
        optimizer.zero_grad()
        # forward pass: calcular la salida para los datos de entrada..
        pred=model(data)
        # calculate the batch loss
        loss=loss_module(pred,target)
        # backpropagation: cálculo de gradientes
        loss.backward()
        # actualizar los parámetros
        optimizer.step()
        # actualizar la cuenta de costos a lo largo de los lotes
        train_loss += loss.item()*data.size(0)
        _, predicted = torch.max(pred.data, 1)


        total += target.size(0)
        train_accuracy += (predicted == target).sum().item()



    train_accuracy = train_accuracy / total
    train_loss = train_loss/len(train_loader.dataset) 


    model.eval() #Ponemos el modelo en modo evaluación.

    # vamos a evaluar el modelo entrenado, calculando predicciones con el conjunto de validación
    
    for data,target in valid_loader:
      output=model(data)
      valid_loss= loss_module(output,target)
      valid_loss += loss.item()*data.size(0)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
    #imprimir estadísticas de entrenamiento, validación y accuracy
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \t train accuracy: {:.6f}'.format(
        i, train_loss, valid_loss, train_accuracy))
    
    #Guardamos el modelo con el menor error de validación.
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'mejor_modelo.pt')
        valid_loss_min = valid_loss


In [17]:
train_model(model,optimizer,criterion,train_data,val_data)

Epoch: 0 	Training Loss: 1.050604 	Validation Loss: 0.038475 	 train accuracy: 0.452542
Validation loss decreased (inf --> 0.038475).  Saving model ...
Epoch: 1 	Training Loss: 0.956086 	Validation Loss: 0.047805 	 train accuracy: 0.544068
Epoch: 2 	Training Loss: 0.931357 	Validation Loss: 0.053040 	 train accuracy: 0.555932
Epoch: 3 	Training Loss: 0.917564 	Validation Loss: 0.056610 	 train accuracy: 0.572881
Epoch: 4 	Training Loss: 0.908560 	Validation Loss: 0.058888 	 train accuracy: 0.584746
Epoch: 5 	Training Loss: 0.901992 	Validation Loss: 0.060205 	 train accuracy: 0.589831
Epoch: 6 	Training Loss: 0.896910 	Validation Loss: 0.060887 	 train accuracy: 0.586441
Epoch: 7 	Training Loss: 0.892852 	Validation Loss: 0.061167 	 train accuracy: 0.581356
Epoch: 8 	Training Loss: 0.889537 	Validation Loss: 0.061194 	 train accuracy: 0.579661
Epoch: 9 	Training Loss: 0.886775 	Validation Loss: 0.061065 	 train accuracy: 0.577966
Epoch: 10 	Training Loss: 0.884430 	Validation Loss: 0.0

6.
Se imprimen los parámetros del mejor modelo.

In [18]:
model.load_state_dict(torch.load('mejor_modelo.pt'))

<All keys matched successfully>

In [19]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data)

fc1.weight tensor([[-2.0780e-01, -7.0553e-02, -2.6968e-01, -2.2490e-01,  3.6765e-02,
         -7.6568e-02, -8.0756e-02, -2.2143e-02, -7.0459e-02,  1.4234e-01,
          9.1919e-02, -1.2659e-01, -6.6352e-02,  2.4918e-01,  1.2892e-01,
         -8.5321e-02,  1.3112e-01,  2.3676e-01, -1.4436e-01, -5.7857e-02],
        [ 3.9667e-02,  2.4754e-02, -1.2540e-01,  1.7153e-02,  8.6064e-02,
         -2.1354e-01, -8.3746e-02,  1.0602e-01,  3.0075e-02, -1.6061e-01,
          3.2718e-02, -6.7988e-02, -2.3838e-02,  2.8118e-01, -1.2420e-01,
          2.4512e-01, -1.6226e-01, -1.5698e-01,  2.3613e-01, -4.0414e-02],
        [ 8.2673e-02,  5.0848e-02,  9.0254e-02,  1.1390e-01, -1.6175e-01,
         -1.4797e-01,  8.8444e-02, -8.6820e-02, -3.5701e-04, -1.4655e-01,
          5.7093e-02,  2.0184e-01, -9.4009e-02,  1.4178e-01,  5.6044e-02,
         -8.4096e-02, -3.8238e-02, -1.7054e-01, -1.5683e-01, -2.1845e-01],
        [-2.4519e-02,  7.1004e-02, -1.4392e-01,  6.8924e-02,  7.9285e-02,
         -2.4765e-02,  1

7.
Se evalua el desempeño del mejor modelo, para esto se define testAccuracy el cual nos da el accuracy y aqui es donde nos indica si el modelo predice bien los datos.

In [20]:
def testAccuracy():
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_data:
            pred, target = data
            # run the model on the test set to predict labels
            outputs = model(pred)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)            
            total += target.size(0)
            accuracy += (predicted == target).sum().item()
               
    # compute the accuracy over all test 
    accuracy = (100 * accuracy / total)
    print("Accuracy of the model:", accuracy)
    return(accuracy)

In [21]:
testAccuracy()

Accuracy of the model: 56.90376569037657


56.90376569037657

El accuracy esta por encima del 50% lo cual es un buen indicador para un clasificador de 3 clases.

8. 

El modelo indica que predice de forma correcta mas del 50% de los datos, sin embargo no predice en su totalidad los datos que se muestren, para este proyecto se manejaron 2 capas y se normalizaron los datos. Para mejorarlo se tendria que pensar en una arquitectura mas compleja que no sean solo capas lineales, se realizo el experimento con distinto tamaños y cantidad de capaz y al mantener capas lineales y al aumentar el tamaño de estas se redujo el accuracy.

9. 
Se elije aleatoriamente un registro del dataset, para esto el modelo recibe un tensor X, por esta razón se genera el registro aleatorio apartir de X y se define como float32 para que no entre en conflicto al momento de ponerlo en model.

In [22]:
from random import randint

reg_aleatorio = torch.tensor(X.values[randint(0,len(X)-1)], dtype= torch.float32)
reg_aleatorio

tensor([ 0.3937, -0.8742,  0.3832,  0.1757, -0.0188, -0.2348, -0.9784, -0.9445,
        -0.0447, -0.8407,  0.2619,  0.3937,  0.2718, -0.5323, -0.0626,  0.7035,
         0.4920,  0.8868,  0.8651,  0.4406])

La función sigmoid muestra la probabilidad entre las 3 clases.

In [23]:
torch.sigmoid(model(reg_aleatorio))

tensor([0.4734, 0.4497, 0.4305], grad_fn=<SigmoidBackward0>)

Se muestra la probabilidad entre indices del tensor 0,1,2 siendo happy->0  angry->1  sad->2 para así predecir su sentimiento.